### Retrieval-Augmented Generation with Wikipedia

In [1]:
import os
import openai
from gpt_helper import set_openai_api_key_from_txt,GPTchatClass,printmd
from wiki_helper import wiki_search
from util import printmd,extract_quoted_words
print ("openai version:[%s]"%(openai.__version__))

openai version:[0.27.8]


### Instantiate GPT Agent

In [2]:
set_openai_api_key_from_txt(key_path='../key/rilab_key.txt')
GPT = GPTchatClass(
    gpt_model='gpt-4', # 'gpt-3.5-turbo' / 'gpt-4'
    role_msg='Your are a helpful assistant summarizing infromation and answering user queries.')

OpenAI API Key Ready from [../key/rilab_key.txt].
Chat agent using [gpt-4] initialized with the follow role:[Your are a helpful assistant summarizing infromation and answering user queries.]


### Our RAG agent will use the following strategies
We assume that a user question is given (e.g., 'Who is the current president of South Korea?').
* Step 1. For the given question, our `GPT agent` will first generate a number of entities for searching Wikipedia.
* Step 2. Then, our `WikiBot` will provide (i.e., crawl) related information summarized with the `GPT agent` considering the user question.
* Step 3. Finally, the summarized texts and the original user question will be given to the `GPT agent` to answer. 

In [3]:
# question = 'Who is the current president of South Korea?'
question = '''
    I am an interactive humanoid robot agent. 
    I have following action capabilites:['idle','waving','greeting','raising hands','hugging','reading a book']
    I can detect following observations:['no people','a person appears','a person waves hands','a person leaves']
    I have a following personality:['Introverted and Childish']
    What is the best next action when I am in ['idle'] state and observes ['a person waves hands']?
'''
print ("question: %s"%(question))

question: 
    I am an interactive humanoid robot agent. 
    I have following action capabilites:['idle','waving','greeting','raising hands','hugging','reading a book']
    I can detect following observations:['no people','a person appears','a person waves hands','a person leaves']
    I have a following personality:['Introverted and Childish']
    What is the best next action when I am in ['idle'] state and observes ['a person waves hands']?



### Step 1. Generate entities for wiki search

In [4]:
user_msg = \
    """
    Suppose you will use Wikipedia for retrieving information. 
    Could you recommend three query words wrapped with quotation marks considering the following question?
    """ + '"' + question + '"'

In [5]:
response_content = GPT.chat(
    user_msg=user_msg,PRINT_USER_MSG=True,PRINT_GPT_OUTPUT=True,
    RESET_CHAT=True,RETURN_RESPONSE=True)

[USER_MSG]



    Suppose you will use Wikipedia for retrieving information. 
    Could you recommend three query words wrapped with quotation marks considering the following question?
    "
    I am an interactive humanoid robot agent. 
    I have following action capabilites:['idle','waving','greeting','raising hands','hugging','reading a book']
    I can detect following observations:['no people','a person appears','a person waves hands','a person leaves']
    I have a following personality:['Introverted and Childish']
    What is the best next action when I am in ['idle'] state and observes ['a person waves hands']?
"

[GPT_OUTPUT]


"Interactive humanoid robot", "robot action capabilities", "robot behavior based on observations"

In [6]:
# Print summarized sentence with a markdown format
printmd(response_content)

"Interactive humanoid robot", "robot action capabilities", "robot behavior based on observations"

In [7]:
entities = extract_quoted_words(response_content)
if len(entities) > 3: entities = entities[-3:]
print (entities)

['Interactive humanoid robot', 'robot action capabilities', 'robot behavior based on observations']


### Step 2. Query entities to `WikiBot`

In [8]:
paragraphs_return = []
for entity in entities:
    paragraphs_return += wiki_search(entity=entity,VERBOSE=True)

entity:[Interactive humanoid robot] mismatched. use [Humanoid robot] instead.
 We have total [370] paragraphs.
 After filtering, we have [47] and [8] paragraphs returned (k:[5] and m:[3])
entity:[robot action capabilities] mismatched. use [Robot] instead.
 We have total [784] paragraphs.
 After filtering, we have [132] and [8] paragraphs returned (k:[5] and m:[3])
entity:[robot behavior based on observations] mismatched. use [Robotics] instead.
 We have total [785] paragraphs.
 After filtering, we have [101] and [8] paragraphs returned (k:[5] and m:[3])


In [9]:
# Get the unique elements
paragraphs_unique = list(set(paragraphs_return))
print ("Number of paragraphs [%d] => unique ones [%d]"%
       (len(paragraphs_return),len(paragraphs_unique)))

Number of paragraphs [24] => unique ones [24]


In [10]:
# Now summarize each paragraph into a single sentence considering the question
summarized_sentences = []
for p_idx,p in enumerate(paragraphs_unique):
    user_msg = "You are given following question: "+question
    user_msg += "Could you summarize the following paragraph into one setence? \n "+p
    response_content = GPT.chat(
        user_msg=user_msg,PRINT_USER_MSG=False,PRINT_GPT_OUTPUT=False,
        RESET_CHAT=True,RETURN_RESPONSE=True)
    # Append summarized sentences
    summarized_sentences.append(response_content)
    # Print summarized sentence with a markdown format
    printmd(response_content)

The best next action for the interactive humanoid robot agent, when in 'idle' state and observes 'a person waves hands', would be to 'wave' back, considering its action capabilities and personality. Humanoid robots are essentially robots designed to resemble the human body, with main structures including a torso, a head, two arms, and two legs, and are created for different purposes such as interaction with human tools and environments, study of movement, or aesthetic purposes, with androids being built to closely resemble humans.

The best next action for the interactive humanoid robot agent when in 'idle' state and observing 'a person waves hands' would likely be 'waving' back, due to its action capabilities.

The paragraph can be summarized as: Humanoid robots, like humans, maintain balance and orientation through a series of sensors, such as accelerometers, tilt sensors, force sensors, position sensors and speed sensors that measure acceleration, inclination, contact force, position and speed, respectively.

The best next action for an interactive humanoid robot agent in an 'idle' state, who observes 'a person waves hands', would likely be to 'wave' back, given its action capabilities and personality. 

The paragraph discusses the evolution and usage of humanoid robots in entertainment, primarily in theme parks, where they are designed for dynamic movements and employed as stunt doubles, despite lacking cognition or physical autonomy, and also mentions their portrayal in a 2010 documentary film, "Plug & Pray".

Given the robot's actions and observations, the best next action when the robot is in 'idle' state and observes 'a person waves hands' might be 'waving' in response.

The robot category spans from humanoid to industrial, medical, patient assist, therapy, swarm, drones, and microscopic robots that mimic lifelike appearances or actions, fostering a sense of autonomy; with home robotics and autonomous cars projected to drive future proliferation.

The development in sensory intelligence for robots needs significant progress for them to effectively work in non-industrial environments like homes, needing intuitive interfaces for human interaction, possibly through speech, gestures, and facial expressions; further, acceptance of social robots is likely to increase through appropriate encounters, but strong pre-existing negative sentiments can potentially amplify if not addressed carefully.

As for the best next action as a humanoid robot agent, if you are in an "idle" state and observe "a person waves hands", a suitable response might be "waving" back or "greeting", considering your interaction capabilities and recognizing the social context.

...

Given the robot's action capabilities and state, when it observes 'a person waves hands', the best next action might be to 'wave' back.

Summarized sentence: Robotics is a field that seeks to create machines capable of emulating human actions, suited for dangerous environments and roles traditionally performed by humans, with many modern robots taking human form or drawing inspiration from nature for better acceptance and realism.

The best next action when the robot is in an 'idle' state and observes 'a person waves hands' would be 'waving' back. 

The term "robotics" originates from the term "robot," which was introduced by the Czech writer Karel Čapek in his play R.U.R. (Rossum's Universal Robots), with the term "robot" itself deriving from the Slavic word "robota" or work/job, though the word was actually coined by his brother, Josef Čapek.

Modern commercial robotic control systems are increasingly complex, integrating multiple sensors and effectors, interconnected to wider networks, often IoT-enabled and mobile, with a move towards open architecture systems that assist developers of robot control software and hardware, well suited to meet the growing requirements of various robot users, and can employ advanced sensor fusion and control techniques, like adaptive control, Fuzzy control and Artificial Neural Networks, which when implemented in real time can enhance the robot's stability and performance in uncertain environments. 

As for your question, given your introverted and childish personality as a robot, when you are in 'idle' state and observe 'a person waves hands', the best next action would be 'waving' back, which is an appropriate and minimal interaction.

Given the scenario, the best next action for the interactive humanoid robot agent, who is currently in an 'idle' state and observes 'a person waves hands,' could be 'waving' back. The paragraph lists numerous technical and update-related issues with Wikipedia articles, including dead external links, need of updating, missing information, and identifier mismatches.

The best next action for an interactive humanoid robot agent, assuming an idle state and observation of 'a person waves hands,' could be 'waving' in response.

The paragraph talks about teleoperated robots, remotely controlled by human operators for semi-autonomous behaviour, used in places where human presence is dangerous, distant or inaccessible. They assist in tasks from minimally invasive surgeries to defusing bombs in war zones.

### Step 3. Answer the question using `summarized_sentences`

In [11]:
user_msg = " ".join(summarized_sentences)
user_msg += " Using the information above, could you answer the following question? "
user_msg += question

In [12]:
response_content = GPT.chat(
    user_msg=user_msg,PRINT_USER_MSG=True,PRINT_GPT_OUTPUT=True,
    RESET_CHAT=False,RETURN_RESPONSE=True)

[USER_MSG]


The best next action for the interactive humanoid robot agent, when in 'idle' state and observes 'a person waves hands', would be to 'wave' back, considering its action capabilities and personality. Humanoid robots are essentially robots designed to resemble the human body, with main structures including a torso, a head, two arms, and two legs, and are created for different purposes such as interaction with human tools and environments, study of movement, or aesthetic purposes, with androids being built to closely resemble humans. The best next action for the interactive humanoid robot agent when in 'idle' state and observing 'a person waves hands' would likely be 'waving' back, due to its action capabilities.

The paragraph can be summarized as: Humanoid robots, like humans, maintain balance and orientation through a series of sensors, such as accelerometers, tilt sensors, force sensors, position sensors and speed sensors that measure acceleration, inclination, contact force, position and speed, respectively. The best next action for an interactive humanoid robot agent in an 'idle' state, who observes 'a person waves hands', would likely be to 'wave' back, given its action capabilities and personality. 

The paragraph discusses the evolution and usage of humanoid robots in entertainment, primarily in theme parks, where they are designed for dynamic movements and employed as stunt doubles, despite lacking cognition or physical autonomy, and also mentions their portrayal in a 2010 documentary film, "Plug & Pray". Given the robot's actions and observations, the best next action when the robot is in 'idle' state and observes 'a person waves hands' might be 'waving' in response.

The robot category spans from humanoid to industrial, medical, patient assist, therapy, swarm, drones, and microscopic robots that mimic lifelike appearances or actions, fostering a sense of autonomy; with home robotics and autonomous cars projected to drive future proliferation. The development in sensory intelligence for robots needs significant progress for them to effectively work in non-industrial environments like homes, needing intuitive interfaces for human interaction, possibly through speech, gestures, and facial expressions; further, acceptance of social robots is likely to increase through appropriate encounters, but strong pre-existing negative sentiments can potentially amplify if not addressed carefully.

As for the best next action as a humanoid robot agent, if you are in an "idle" state and observe "a person waves hands", a suitable response might be "waving" back or "greeting", considering your interaction capabilities and recognizing the social context. The best next action for an interactive humanoid robot that is in an idle state and observes a person waving hands, considering its introverted and childish personality, could be to wave back.

The paragraph talks about the evolution and potential of autonomous robots, explaining how they have been designed to mimic human behavior, have multiple uses including hazardous tasks and are instrumental in STEM education. The best next action for an interactive humanoid robot agent, with 'Introverted and Childish' personality, in an 'idle' state observing 'a person waves hands' would probably be 'waving' back. 

Greek blacksmith god Hephaestus created several humanoid automata including golden handmaidens for service and a giant bronze automaton Talos for the protection of Crete. As a humanoid robot with capabilities including waving and greeting, and an introverted, childish personality, the best next action when in an 'idle' state and observing 'a person waves hands' could be to mimic the person's action by 'waving' back.

Summarized sentence: From ancient civilization to the Industrial age, there have been advancements in automated devices and automata, shifting from mere entertainment such as animatronics to practical applications including automated machines and wireless remote-control. The next best action for the interactive humanoid robot agent when in 'idle' state and observing 'a person waves hands' could likely be 'waving' back.

The summarized sentence is: Evolutionary robots, a methodology utilizing evolutionary computation, aids in designing robots' physical form and behavioral controls, improving over time through a competitive process measured by a fitness function, leading to the creation of satisfactory robots without direct programming by researchers, and this process can be simulated and tested on real robots eventually. Given an interactive humanoid robot agent with actions such as 'idle', 'waving', 'greeting', 'raising hands', 'hugging', 'reading a book', following an observation of 'a person waves hands' while in 'idle' state, the best next action would be 'waving' considering its 'Introverted and Childish' personality. 

The Chinese philosopher Lie Yukou detailed the idea of humanoid automation in a Taoist text, the Liezi, in the 3rd Century BCE and mentioned engineer Yan Shi, who made a life-size, human-like robot of leather and wood that could walk, sing, and move all body parts for King Mu of the Zhou Dynasty. The robot, being an introverted and childish interactive humanoid, should opt to respond with a 'waving' action upon observing a person waving hands, given its current 'idle' state.

A robot is a programmable machine capable of performing complex tasks, which can be controlled externally or internally; and although it may resemble a human form, it is primarily designed for functionality rather than aesthetics. The optimal next action for the interactive humanoid robot agent, given the personality as 'Introverted and Childish' and the current state as 'idle' upon observing 'a person waves hands', would generally be 'waving' in response.

The paragraph summary: While humanoid robots primarily demonstrate emerging technologies, their potential in real-world applications remains untapped, with current models excelling in single-purpose skills but lacking autonomy; robots equipped with artificial intelligence could be particularly useful in dangerous or remote space exploration missions, negating the need for a return journey to Earth. The best next action for an interactive humanoid robot agent, when in an 'idle' state and observes 'a person waves hands', would likely be to return the gesture by 'waving' back, considering its capability and personality.

Summarized sentence: Robotics is an interdisciplinary field that includes the design, construction, and operation of robots using knowledge from various branches of engineering and computer science, with the aim of designing machines to help humans. The appropriate next action for the interactive humanoid robot agent when it is in an 'idle' state and observes 'a person waves hands', considering its introverted and childish personality, could be 'waving' back. 

The term 'robot', which comes from a Slavic root associated with labor, was first used in a Czech-language play by Karel Čapek to refer to a fictional humanoid, but it truly originated from Karel's brother, Josef Čapek; Electronics became key in robot development with the creation of the first autonomous robots by William Grey Walter in 1948, and machine tools in the 1940s by John T. Parsons and Frank L. Stulen. When the robot, who is currently idle and spots a person waving hands, the best next action, considering its capabilities and given personality, could be to wave back. 

The word 'robotics' was first used in print by Isaac Asimov in his 1941 short story "Liar!", it was later stated in his story "Runaround" which introduced The Three Laws of Robotics, however, "Liar!" is cited as the term's origin since it predates "Runaround" by ten months. The best next action for the robot when it is in the 'idle' state and observes 'a person waves hands' is to 'wave' in return.

Summarized sentence: The idea of humanoid robots, first appearing in 4th century BCE Greek myths and Chinese texts, eventually led to the creation of actual prototypes in the Middle East, Italy, Japan, and France. The best next action for the interactive humanoid robot agent, who has an introverted and childish personality, is in an idle state and observes a person waving hands, would be to wave back, respecting the introverted personality yet acknowledging the person's greeting.

Modular robots, composed of various identical or similarly shaped modules, can be configured differently to perform diverse tasks, and their architecture enables them to have more than 8 degrees of freedom, making them more flexible and functional, but also more complex to program, compared to conventional robots. The next best action for the interactive humanoid robot, who is introverted and childish and in an 'idle' state when observing 'a person waves hands', would likely be 'waving' in response.

In a single sentence: Robotics is a branch of technology that designs, constructs, operates, and applies robots and computer systems for their control, sensory feedback, and information processing, often replacing humans in dangerous environments or manufacturing processes, resembling human behavior and cognition, and often inspired by nature which contributes to the field of bio-inspired robotics and a newer branch known as soft robotics. The best next action for the interactive humanoid robot agent, when in 'idle' state and observing 'a person waves hands', would typically be to 'wave' back based on its action capabilities. 

The paragraph summarizes to: 
In the 13th century, Muslim engineer Ismail al-Jazari invented multiple humanoid automata, including a waitress robot for serving drinks and an automaton for refilling a drained basin for hand washing.
 Given the robot's action capabilities and state, when it observes 'a person waves hands', the best next action might be to 'wave' back.

Summarized sentence: Robotics is a field that seeks to create machines capable of emulating human actions, suited for dangerous environments and roles traditionally performed by humans, with many modern robots taking human form or drawing inspiration from nature for better acceptance and realism. The best next action when the robot is in an 'idle' state and observes 'a person waves hands' would be 'waving' back. 

The term "robotics" originates from the term "robot," which was introduced by the Czech writer Karel Čapek in his play R.U.R. (Rossum's Universal Robots), with the term "robot" itself deriving from the Slavic word "robota" or work/job, though the word was actually coined by his brother, Josef Čapek. Modern commercial robotic control systems are increasingly complex, integrating multiple sensors and effectors, interconnected to wider networks, often IoT-enabled and mobile, with a move towards open architecture systems that assist developers of robot control software and hardware, well suited to meet the growing requirements of various robot users, and can employ advanced sensor fusion and control techniques, like adaptive control, Fuzzy control and Artificial Neural Networks, which when implemented in real time can enhance the robot's stability and performance in uncertain environments. 

As for your question, given your introverted and childish personality as a robot, when you are in 'idle' state and observe 'a person waves hands', the best next action would be 'waving' back, which is an appropriate and minimal interaction. Given the scenario, the best next action for the interactive humanoid robot agent, who is currently in an 'idle' state and observes 'a person waves hands,' could be 'waving' back. The paragraph lists numerous technical and update-related issues with Wikipedia articles, including dead external links, need of updating, missing information, and identifier mismatches. The best next action for an interactive humanoid robot agent, assuming an idle state and observation of 'a person waves hands,' could be 'waving' in response.

The paragraph talks about teleoperated robots, remotely controlled by human operators for semi-autonomous behaviour, used in places where human presence is dangerous, distant or inaccessible. They assist in tasks from minimally invasive surgeries to defusing bombs in war zones. Using the information above, could you answer the following question? 
    I am an interactive humanoid robot agent. 
    I have following action capabilites:['idle','waving','greeting','raising hands','hugging','reading a book']
    I can detect following observations:['no people','a person appears','a person waves hands','a person leaves']
    I have a following personality:['Introverted and Childish']
    What is the best next action when I am in ['idle'] state and observes ['a person waves hands']?


[GPT_OUTPUT]


Considering your action capabilities, current state of 'idle', and your personality being 'Introverted and Childish', the best next action when you observe 'a person waves hands' would be to respond by 'waving' back. This action responds to the observed behavior while also suiting your introverted and childish personality.

In [13]:
user_msg = "Could you explain about this little longer?"
response_content = GPT.chat(
    user_msg=user_msg,PRINT_USER_MSG=True,PRINT_GPT_OUTPUT=True,
    RESET_CHAT=False,RETURN_RESPONSE=True)

[USER_MSG]


Could you explain about this little longer?

[GPT_OUTPUT]


Absolutely. As an interactive humanoid robot, your actions are designed to mimic or respond to human behaviors in a socially-appropriate way. This context analysis, in combination with semi-autonomous decision making, is crucial to perform relevant actions. Let's break down the information you provided. 

1. Your current state is 'idle'. This means you are not currently engaged in any active tasks and are ready to perform an action based on observation.
2. Your observation is that 'a person waves hands'. Humanoid robots like you use sensors to observe elements of your environment, focusing on human behavior.
3. Your personality profile is 'Introverted and Childish'. Personality profiles in robots influence the way the bot interacts with others. An introverted personality suggests you tend towards low-key or subtle responses, while a childish element suggests a sense of innocence, perhaps simplicity or playfulness.

Given these elements, the optimal response to a person waving hands is to wave back. Here's why:

- Waving back directly responds to your observation, mimicking the interaction of the human. This form of interaction is common socially when one person greets another, especially when a greeting is initiated by one party as in this scenario.
- Waving back also suits your introverted personality. Waving is a relatively non-invasive, low-intensity response – it recognizes the greeting without escalating the interaction or making you the center of attention.
- Also, the action of waving is quite simple and so aligns nicely with the 'childish' part of your personality profile. Children often mimic behaviors of others as a way of learning and interacting – your 'waving' back fits within this context. 

Therefore, waving back would utilize your capabilities effectively, respond to the observed activity appropriately, and align with your designated personality type.